Instalando pacotes e bibliotecas necessárias

In [1]:
!pip install numpy-financial
!pip install XlsxWriter
!pip install openpyxl

import pandas as pd
import numpy as np
import numpy_financial as npf
import statsmodels.api as sm

Definindo variáveis de entrada

In [23]:
# Arquivo de saída
arquivo = 'UFV 1.xlsx'

# Definindo os dados iniciais
ano_inicial = 2024
periodo = 30
taxa_inflacao_anual = [0.0392, 0.035, 0.035, 0.035, 0.035]
vida_util_depreciacao = 15  # Vida útil para cálculo de depreciação (anos)
preco_he = 1200   # R$/He
area_por_MW = 2   # Em He

# Modelagem financeira
wacc = 0.15
anos_tir = [5, 10, 15, 20, 25, 30]
index_despesa = 1

# Definindo a UFV
potencia_dc = 1.0   # kW
potencia_ac = 1.3   # kWp
producao_especifica = 150   # Produção em kWh/kWp/mês
tusd = 20.0   # R$/kW

# Definindo CAPEX
preco_kW = 4000   # R$/kWp

# Definindo receita a partir do preço da energia
preco_energia = 650   # Preço de venda da energia em R$/MWh
geracao_anual = 2160    # Input de geração anual vinda do PVSyst em MWh
taxa_comissao = 0.06  # Taxa de comissão do vendedor (6%)


Definindo função para gerar DRE e Fluxo de Caixa

In [24]:
def calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                             potencia_dc, area_por_MW, tusd, preco_kW, preco_energia, geracao_anual,
                             index_despesa, taxa_comissao, vida_util_depreciacao, wacc, excel_filename, teste):

    #Calculando os valores iniciais:
    area = potencia_ac * area_por_MW    # Área necessária em He
    investimento_inicial = potencia_ac * preco_kW * 1000
    receita_inicial = preco_energia * geracao_anual / 12
    aluguel = area * preco_he
    seguro = investimento_inicial * 0.7 * 0.008 / 12
    manutencao = investimento_inicial * 0.007 / 12
    demanda_contratada = tusd * potencia_dc


    # Criando listas para armazenar os dados de fluxo de caixa
    fluxo_caixa = []
    entrada = []
    saida = []

    # Criando listas para armazenar os dados de DRE mensal
    anos_mensal = []
    meses_mensal = []
    receitas_brutas_mensal = []
    impostos_sobre_receita_mensal = []
    deducoes_sobre_receita_mensal = []
    receitas_liquidas_mensal = []
    despesas_operacionais_mensal = []
    outras_despesas_mensal = []
    despesas_financeiras_mensal = []
    irpj_mensal = []
    csll_mensal = []
    resultado_exercicio_mensal = []
    lucro_bruto_mensal = []
    receitas_financeiras_mensal = []
    depreciacao_amortizacao_mensal = []
    lucro_antes_ir_csll_mensal = []

    # Criando listas para armazenar os dados de DRE anual
    anos_anual = []
    meses_anual = []
    receitas_brutas_anual = []
    impostos_sobre_receita_anual = []
    deducoes_sobre_receita_anual = []
    receitas_liquidas_anual = []
    despesas_operacionais_anual = []
    outras_despesas_anual = []
    despesas_financeiras_anual = []
    irpj_anual = []
    csll_anual = []
    resultado_exercicio_anual = []

    # Preenchendo os dados para vários anos e meses
    for ano, taxa_inflacao in zip(range(ano_inicial, ano_inicial + periodo), taxa_inflacao_anual * 6):
        for mes in range(1, 13):
            meses_passados = (ano - ano_inicial) * 12 + mes
            anos_mensal.append(ano)
            meses_mensal.append(mes)
            meses_anual.append(mes)

            inflacao_acumulada = np.prod([1 + taxa for taxa in taxa_inflacao_anual[:meses_passados // 12]]) - 1

            receitas_brutas_mensal_atual = receita_inicial * (1 + inflacao_acumulada) if meses_passados >= 10 else 0
            receitas_brutas_mensal.append(receitas_brutas_mensal_atual)

            impostos_sobre_receita_mensal_atual = receitas_brutas_mensal_atual * 0  # Exemplo de taxa de imposto
            impostos_sobre_receita_mensal.append(impostos_sobre_receita_mensal_atual)

            deducoes_sobre_receita_mensal_atual = receitas_brutas_mensal_atual * taxa_comissao
            deducoes_sobre_receita_mensal.append(deducoes_sobre_receita_mensal_atual)

            receitas_liquidas_mensal_atual = receitas_brutas_mensal_atual - impostos_sobre_receita_mensal_atual - deducoes_sobre_receita_mensal_atual
            receitas_liquidas_mensal.append(receitas_liquidas_mensal_atual)

            despesas_op_mensal = sum([valor * (1 + inflacao_acumulada) for valor in [aluguel, seguro, manutencao, demanda_contratada]]) if meses_passados >= 10 else 0
            despesas_operacionais_mensal_atual = despesas_op_mensal
            despesas_operacionais_mensal.append(despesas_operacionais_mensal_atual)

            outras_despesas_mensal_atual = 0  # Substitua isso pelos valores reais, se houver outras despesas
            outras_despesas_mensal.append(outras_despesas_mensal_atual)

            lucro_bruto_mensal_atual = receitas_liquidas_mensal_atual - despesas_operacionais_mensal_atual - outras_despesas_mensal_atual
            lucro_bruto_mensal.append(lucro_bruto_mensal_atual)

            deprec_amort = 0  # Exemplo de depreciação/amortização
            depreciacao_amortizacao_mensal_atual = deprec_amort
            depreciacao_amortizacao_mensal.append(depreciacao_amortizacao_mensal_atual)

            receitas_financeiras_mensal_atual = 0  # Substitua isso pelos valores reais, se houver receitas financeiras
            receitas_financeiras_mensal.append(receitas_financeiras_mensal_atual)

            despesas_financeiras_mensal_atual = 0  # Substitua isso pelos valores reais, se houver receitas financeiras
            despesas_financeiras_mensal.append(despesas_financeiras_mensal_atual)

            lucro_antes_ir_csll_mensal_atual = lucro_bruto_mensal_atual - depreciacao_amortizacao_mensal_atual + receitas_financeiras_mensal_atual
            lucro_antes_ir_csll_mensal.append(lucro_antes_ir_csll_mensal_atual)

            irpj_mensal_atual = 0  # Exemplo de cálculo de IRPJ
            irpj_mensal.append(irpj_mensal_atual)

            csll_mensal_atual = 0  # Exemplo de cálculo de CSLL
            csll_mensal.append(csll_mensal_atual)

            # Calculando resultado do exercício mensal
            resultado_exercicio_mensal_atual = lucro_antes_ir_csll_mensal_atual - irpj_mensal_atual - csll_mensal_atual
            resultado_exercicio_mensal.append(resultado_exercicio_mensal_atual)

            # Fluxo de caixa
            if meses_passados < 6:
                entrada_mensal = 0
                saida_mensal = investimento_inicial / 6
                fluxo_caixa.append(entrada_mensal - saida_mensal)
                entrada.append(entrada_mensal)
                saida.append(saida_mensal)
            else:
                entrada_mensal = receitas_brutas_mensal_atual + receitas_financeiras_mensal_atual
                saida_mensal = impostos_sobre_receita_mensal_atual + deducoes_sobre_receita_mensal_atual + despesas_operacionais_mensal_atual + outras_despesas_mensal_atual + despesas_financeiras_mensal_atual + irpj_mensal_atual + csll_mensal_atual
                fluxo_caixa.append(entrada_mensal - saida_mensal)
                entrada.append(entrada_mensal)
                saida.append(saida_mensal)

        # Atualizando os valores anuais
        anos_anual.append(ano)
        receitas_brutas_anual.append(sum(receitas_brutas_mensal))
        impostos_sobre_receita_anual.append(sum(impostos_sobre_receita_mensal))
        deducoes_sobre_receita_anual.append(sum(deducoes_sobre_receita_mensal))
        receitas_liquidas_anual.append(sum(receitas_liquidas_mensal))
        despesas_operacionais_anual.append(sum(despesas_operacionais_mensal))
        outras_despesas_anual.append(sum(outras_despesas_mensal))
        despesas_financeiras_anual.append(sum(despesas_financeiras_mensal))
        irpj_anual.append(sum(irpj_mensal))
        csll_anual.append(sum(csll_mensal))
        resultado_exercicio_anual.append(sum(resultado_exercicio_mensal))

    # Criando um DataFrame Pandas para a DRE mensal
    dados_dre_mensal = {
        'Ano': anos_mensal,  #ok
        'Mês': meses_mensal, #ok
        'Receita Operacional Bruta': receitas_brutas_mensal, #ok
        'Impostos sobre Receita': impostos_sobre_receita_mensal,
        'Deduções sobre Receita': deducoes_sobre_receita_mensal,
        'Receita Líquida': receitas_liquidas_mensal,
        'Despesas Operacionais': despesas_operacionais_mensal,
        'Outras Despesas': outras_despesas_mensal,
        'Lucro Bruto': lucro_bruto_mensal,
        'Depreciação/Amortização': depreciacao_amortizacao_mensal,
        'EBIT': lucro_antes_ir_csll_mensal,  # Considerando EBIT como Lucro Bruto sem depreciação
        'Receitas Financeiras': receitas_financeiras_mensal,
        'Despesas Financeiras': despesas_financeiras_mensal,
        'Lucro antes do IR e CSLL': lucro_antes_ir_csll_mensal,
        'IRPJ': irpj_mensal,
        'CSLL': csll_mensal,
        'Resultado do Exercício': resultado_exercicio_mensal,
    }

    df_dre_mensal = pd.DataFrame(dados_dre_mensal)

    # Criando um DataFrame Pandas para a DRE anual
    dados_dre_anual = {
        'Ano': anos_anual,
        'Receita Operacional Bruta': receitas_brutas_anual,
        'Impostos sobre Receita': impostos_sobre_receita_anual,
        'Deduções sobre Receita': deducoes_sobre_receita_anual,
        'Receita Líquida': receitas_liquidas_anual,
        'Despesas Operacionais': despesas_operacionais_anual,
        'Outras Despesas': outras_despesas_anual,
        'Despesas Financeiras': despesas_financeiras_anual,
        'IRPJ': irpj_anual,
        'CSLL': csll_anual,
        'Resultado do Exercício': resultado_exercicio_anual,
    }

    df_dre_anual = pd.DataFrame(dados_dre_anual)

    # Criando um DataFrame Pandas para o fluxo de caixa mensal
    df_fluxo_caixa_mensal = pd.DataFrame({
        'Ano': anos_mensal,
        'Mês': meses_mensal,
        'Entradas': entrada,
        'Saídas': saida,
        'Resultado': fluxo_caixa,
    })

    if teste == 0:

      # Salvando todos os DataFrames no mesmo arquivo Excel, em abas diferentes
      with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
          df_dre_mensal.to_excel(writer, sheet_name='DRE_Mensal', index=False)
          df_dre_anual.to_excel(writer, sheet_name='DRE_Anual', index=False)
          df_fluxo_caixa_mensal.to_excel(writer, sheet_name='Fluxo_Caixa_Mensal', index=False)

    return df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal


Definindo função para calcular TIR e VPL

In [25]:
def calcular_tir_vpl(df_fluxo_caixa, anos, ano_inicial, wacc, excel_filename, teste):

    results = {'Ano': [], 'TIR': [], 'VPL': []}

    for ano in anos:
        # Filtra o DataFrame para o ano específico
        df_ano = df_fluxo_caixa[df_fluxo_caixa['Ano'] < (ano_inicial + ano)]

        # Calcula a Taxa Interna de Retorno (TIR)
        tir = npf.irr(df_ano['Resultado'])
        tir = ((1 + tir) ** 12 -1)*100

        # Calcula o Valor Presente Líquido (VPL)
        taxa_desconto = ((1 + wacc) ** (1/12) -1)
        vpl = npf.npv(0.001, df_ano['Resultado']) / 1000000

        # Armazena os resultados
        results['Ano'].append(ano)
        results['TIR'].append(tir)
        results['VPL'].append(vpl)

    # Cria um DataFrame com os resultados
    df_resultados = pd.DataFrame(results)

    if teste == 0:
      # Abre o arquivo Excel existente
      with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a') as writer:
          # Adiciona o DataFrame dos resultados em uma nova aba
          df_resultados.to_excel(writer, sheet_name='TIR e VPL', index=False)

    return df_resultados


In [26]:
# Exemplo de uso da função
teste = 0
df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, preco_kW, preco_energia, geracao_anual,
                                                                              index_despesa, taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)

In [27]:
teste = 0
resultados = calcular_tir_vpl(df_fluxo_caixa_mensal, anos_tir, ano_inicial, wacc, arquivo, teste)

# Exibir os resultados
print(resultados)


   Ano        TIR        VPL
0    5  10.577619   1.136093
1   10  27.725309   7.770150
2   15  30.288758  14.018057
3   20  30.840287  19.902290
4   25  30.974174  25.444019
5   30  31.008096  30.663179


Agora tenho que criar uma análise de sensibilidade.

Variar: preço de energia,

In [7]:
'''
def sensibilidade(variavel):

  teste = 1
  variacao = [-15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]

  for i in variacao:

    geracao_anual = variavel * (1 + i /100)
    print(geracao_anual)

    df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, preco_kW, preco_energia, geracao_anual,
                                                                              taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)


    # Filtra o DataFrame para o ano específico
    df_25 = df_fluxo_caixa_mensal[df_fluxo_caixa_mensal['Ano'] < (ano_inicial + 25)]

    # Calcula a Taxa Interna de Retorno (TIR)
    tir = npf.irr(df_25['Resultado'])
    tir = ((1 + tir) ** 12 -1)*100

    print(tir)



SyntaxError: ignored

In [ ]:
#sensibilidade(geracao_anual)

In [28]:
  teste = 1
  variacao = [-15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]

  for i in variacao:

    variavel = geracao_anual
    variavel *= (1 + i /100)
    print(geracao_anual)


    df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, preco_kW, preco_energia, variavel,
                                                                              index_despesa, taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)


    # Filtra o DataFrame para o ano específico
    df_25 = df_fluxo_caixa_mensal[df_fluxo_caixa_mensal['Ano'] < (ano_inicial + 25)]

    # Calcula a Taxa Interna de Retorno (TIR)
    tir = npf.irr(df_25['Resultado'])
    tir = ((1 + tir) ** 12 -1)*100

    print(tir)

2160
26.075271813357183
2160
27.717712963524853
2160
29.35016158509893
2160
30.97417395643962
2160
32.590990345347
2160
34.201609093137364
2160
35.806841928976965
2160
37.407355593892724
2160
39.00370337304378
2160
40.59634912742129
2160
42.18568571045538
2160
43.77204915738457
2160
46.936980200552036


In [29]:
  teste = 1
  variacao = [-15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]

  for i in variacao:

    variavel = preco_energia
    variavel *= (1 + i /100)
    print(variavel)

    df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, preco_kW, variavel, geracao_anual,
                                                                              index_despesa, taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)


    # Filtra o DataFrame para o ano específico
    df_25 = df_fluxo_caixa_mensal[df_fluxo_caixa_mensal['Ano'] < (ano_inicial + 25)]

    # Calcula a Taxa Interna de Retorno (TIR)
    tir = npf.irr(df_25['Resultado'])
    tir = ((1 + tir) ** 12 -1)*100

    print(tir)

552.5
26.075271813357183
585.0
27.717712963524853
617.5
29.35016158509893
650.0
30.97417395643962
682.5
32.590990345347
715.0000000000001
34.20160909313703
747.4999999999999
35.80684192899073
780.0
37.407355593892724
812.5
39.00370337304378
845.0
40.59634912742129
877.5000000000001
42.18568571046275
909.9999999999999
43.7720491573961
975.0
46.936980200552036


In [30]:
  teste = 1
  variacao = [-15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]

  for i in variacao:

    variavel = preco_kW
    variavel *= (1 + i /100)
    print(variavel)

    df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, variavel, preco_energia, geracao_anual,
                                                                              index_despesa, taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)


    # Filtra o DataFrame para o ano específico
    df_25 = df_fluxo_caixa_mensal[df_fluxo_caixa_mensal['Ano'] < (ano_inicial + 25)]

    # Calcula a Taxa Interna de Retorno (TIR)
    tir = npf.irr(df_25['Resultado'])
    tir = ((1 + tir) ** 12 -1)*100

    print(tir)

3400.0
36.49343707422887
3600.0
34.45682304354807
3800.0
32.627420348832594
4000.0
30.97417395643962
4200.0
29.47192907433216
4400.0
28.1000942231898
4600.0
26.841652708622533
4800.0
25.68242070838933
5000.0
24.610482882719566
5200.0
23.61575773890472
5400.0
22.689659164490173
5600.0
21.824830150474828
6000.0
20.254471666518125


In [36]:
  teste = 1
  variacao = [-15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]

  for i in variacao:

    variavel = index_despesa
    variavel *= (1 + i /100)
    print(variavel)

    df_dre_mensal, df_dre_anual, df_fluxo_caixa_mensal = calcular_dre_fluxo_caixa(ano_inicial, periodo, taxa_inflacao_anual, preco_he, potencia_ac,
                                                                              potencia_dc, area_por_MW, tusd, preco_kW, preco_energia, geracao_anual,
                                                                              1, taxa_comissao, vida_util_depreciacao, wacc, arquivo, teste)


    # Filtra o DataFrame para o ano específico
    df_25 = df_fluxo_caixa_mensal[df_fluxo_caixa_mensal['Ano'] < (ano_inicial + 25)]

    # Calcula a Taxa Interna de Retorno (TIR)
    tir = npf.irr(df_25['Resultado'])
    tir = ((1 + tir) ** 12 -1)*100

    print(tir)

0.85
30.97417395643962
0.9
30.97417395643962
0.95
30.97417395643962
1.0
30.97417395643962
1.05
30.97417395643962
1.1
30.97417395643962
1.15
30.97417395643962
1.2
30.97417395643962
1.25
30.97417395643962
1.3
30.97417395643962
1.35
30.97417395643962
1.4
30.97417395643962
1.5
30.97417395643962
